***AYUDANTIA 15***

Importar librerias

In [1]:
import numpy as np
import sympy as sp


**Forma Matricial**

Funciones

In [ ]:
def prob_equilibrio(matriz_tasas):
    m_final = []

    # creamos y rellenamos matriz final
    # (matriz cuadrada llena de ceros que se llenará con las ecuaciones de equilibrio para cada estado)

    for i in range(len(matriz_tasas)):
        row = []
        for j in range(len(matriz_tasas)):
            row.append(0)
        m_final.append(row)

    t = 0
    #creamos matriz que representa ecuaciones de equilibrio
    for fila in matriz_tasas:
        #obtenemos tasas de salida
        sale = sum(fila)
        #ponemos la tasa de salida en la fila correspondiente al nodo
        m_final[t][t] = sale

        for h in range(len(matriz_tasas)):
            #si estamos en un nodo distinto y llega algo lo ponemos negativo (tasas de llegada)
            if h!= t and matriz_tasas[h][t] != 0:
                m_final[t][h] = -matriz_tasas[h][t]
        t+=1 

    #Agregar ecuacuion de normalizacion (sum P_i = 1)
    matriz_final = np.vstack((np.matrix(m_final),[[1]*len(m_final)]))
    vector_p = np.array([[0]]*(len(matriz_final)-1)+[[1]]).reshape(len(matriz_final))


    #Resolver sistema 
    P = np.linalg.lstsq(matriz_final,vector_p,rcond=-1)[0]

    return P


def imprimir(P):
    k = 0
    for fila in P:
        print(f"P_{k} = {fila}")
        k += 1

def esperanza(P):
    k = 0
    esp = 0
    for fila in P:
        esp += k* fila
        k+= 1
    return esp

Parte 1 

In [3]:
# Definir parametros 
K = 5
gam = 2
thet = 1

# Crear matriz de tasas 
# (Cada entrada [i][j] representa la tasa a la que el sistema pasa del estado i al estado j)

matriz_tasas1 = [
    [0,(K+1)*gam,0,0,0,0,0],
    [thet,0,K*gam,0,0,0,0],
    [0,2*thet,0,(K-1)*gam,0,0,0],
    [0,0,3*thet,0,(K-2)*gam,0,0],
    [0,0,0,4*thet,0,(K-3)*gam,0],
    [0,0,0,0,5*thet,0,(K-4)*gam],
    [0,0,0,0,0,6*thet,0]
    ]


prob_parte_1 = prob_equilibrio(matriz_tasas1)
imprimir(prob_parte_1)
esperanza = esperanza(prob_parte_1)
print(f"Esperanza: {esperanza}")


P_0 = 0.0013717421124828497
P_1 = 0.016460905349794185
P_2 = 0.08230452674897101
P_3 = 0.21947873799725628
P_4 = 0.32921810699588455
P_5 = 0.2633744855967078
P_6 = 0.0877914951989025
Esperanza: 3.9999999999999973


Parte 2 (propuesto)

In [4]:

Lambda = 1
mu = 2*Lambda


matriz_tasas2 = [
    [0,10*mu,0,10*mu,0,0,0,0,0],
    [Lambda,0,5*mu,0,10*mu,0,0,0,0],
    [0,2*Lambda,0,0,0,10*mu,0,0,0],
    [Lambda,0,0,0,10*mu,0,5*mu,0,0],
    [0,Lambda,0,Lambda,0,mu,0,mu,0],
    [0,0,Lambda,0,2*Lambda,0,0,0,mu],
    [0,0,0,2*Lambda,0,0,0,10*mu,0],
    [0,0,0,0,2*Lambda,0,Lambda,0,mu],
    [0,0,0,0,0,2*Lambda,0,2*Lambda,0]
    ]

prob_parte_2 = prob_equilibrio(matriz_tasas2)
imprimir(prob_parte_2)

P_0 = 0.000413736036408771
P_1 = 0.00827472072817541
P_2 = 0.014894497310715814
P_3 = 0.008274720728175448
P_4 = 0.21845262722383127
P_5 = 0.24493173355399273
P_6 = 0.014894497310715763
P_7 = 0.24493173355399256
P_8 = 0.2449317335539927


**Forma Sympy**

In [5]:
def resolver_sistema(K, gamma, theta):

    # Definir las variables de probabilidades 
    P = [sp.symbols(f'P{i}') for i in range(K+2)]
    
    # Crear la lista de ecuaciones
    ecuaciones = []
    diccionario_ecuaciones = {}

    # Primera ecuación: P0(K + 1)γ = θP1
    ecuaciones.append(P[0]*(K+1)*gamma - theta* P[1])
    diccionario_ecuaciones[0] = P[0]*(K+1)*gamma - theta* P[1]

    # Ecuaciones recursivas para i ∈ {1, K}
    for i in range(1, K):
        ecuacion = P[i]*((K-i+1)*gamma + i*theta) - P[i-1]*(K-i+2)*gamma - (i+1)*theta*P[i+1]
        ecuaciones.append(ecuacion)
        diccionario_ecuaciones[i] = ecuacion

    # Última ecuación: PK+1(K + 1)θ = γPK
    ecuacion = P[K+1]*(K+1)*theta - gamma * P[K]
    ecuaciones.append(ecuacion)
    diccionario_ecuaciones[K+1] = ecuacion

    # Ecuación de normalización: suma de las probabilidades = 1
    ecuacion = sum(P) - 1
    ecuaciones.append(ecuacion)
    diccionario_ecuaciones['normalizacion'] = ecuacion
    
    # Resolver el sistema de ecuaciones
    soluciones = sp.solve(ecuaciones, P)

    # Guardar soluciones en el diccionario
    diccionario_soluciones = {i: soluciones[P[i]] for i in range(K+2)}

    return diccionario_ecuaciones, diccionario_soluciones


In [6]:

K = 5
gamma = 2
theta = 1

diccionario_ecuaciones, diccionario_soluciones = resolver_sistema(K,gamma,theta)

# Imprimir el diccionario de ecuaciones
print("Ecuaciones planteadas:")
for key, eq in diccionario_ecuaciones.items():
    print(f"Ecuación {key}: {eq}")

# Imprimir las soluciones
print("\nSoluciones del sistema:")
for key, sol in diccionario_soluciones.items():
    print(f"P{key} = {sol}")


# Imprimir las soluciones decimales
print("\nSoluciones del sistema (decimal):")
for key, sol in diccionario_soluciones.items():
    print(f"P{key} = {sol.evalf()}")


Ecuaciones planteadas:
Ecuación 0: 12*P0 - P1
Ecuación 1: -12*P0 + 11*P1 - 2*P2
Ecuación 2: -10*P1 + 10*P2 - 3*P3
Ecuación 3: -8*P2 + 9*P3 - 4*P4
Ecuación 4: -6*P3 + 8*P4 - 5*P5
Ecuación 6: -2*P5 + 6*P6
Ecuación normalizacion: P0 + P1 + P2 + P3 + P4 + P5 + P6 - 1

Soluciones del sistema:
P0 = 1/729
P1 = 4/243
P2 = 20/243
P3 = 160/729
P4 = 80/243
P5 = 64/243
P6 = 64/729

Soluciones del sistema (decimal):
P0 = 0.00137174211248285
P1 = 0.0164609053497942
P2 = 0.0823045267489712
P3 = 0.219478737997257
P4 = 0.329218106995885
P5 = 0.263374485596708
P6 = 0.0877914951989026
